In [6]:
from data_generator.object_detection_2d_data_generator import DataGenerator

In [9]:
train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)

In [11]:
train_dataset.parse_wider_mat(["/mnt/Data/Workspace/ml/datasets/WIDER/WIDER_train/images/"], 
                             ["/mnt/Data/Workspace/ml/datasets/WIDER/wider_face_split/wider_face_train.mat"])

In [12]:
train_dataset.get_dataset_size()

12880

In [13]:
filenames, labels, image_ids, __ = train_dataset.get_dataset()

In [14]:
filenames

['/mnt/Data/Workspace/ml/datasets/WIDER/WIDER_train/images/0--Parade/0_Parade_marchingband_1_849.jpg',
 '/mnt/Data/Workspace/ml/datasets/WIDER/WIDER_train/images/0--Parade/0_Parade_Parade_0_904.jpg',
 '/mnt/Data/Workspace/ml/datasets/WIDER/WIDER_train/images/0--Parade/0_Parade_marchingband_1_799.jpg',
 '/mnt/Data/Workspace/ml/datasets/WIDER/WIDER_train/images/0--Parade/0_Parade_marchingband_1_117.jpg',
 '/mnt/Data/Workspace/ml/datasets/WIDER/WIDER_train/images/0--Parade/0_Parade_marchingband_1_778.jpg',
 '/mnt/Data/Workspace/ml/datasets/WIDER/WIDER_train/images/0--Parade/0_Parade_Parade_0_343.jpg',
 '/mnt/Data/Workspace/ml/datasets/WIDER/WIDER_train/images/0--Parade/0_Parade_marchingband_1_205.jpg',
 '/mnt/Data/Workspace/ml/datasets/WIDER/WIDER_train/images/0--Parade/0_Parade_Parade_0_106.jpg',
 '/mnt/Data/Workspace/ml/datasets/WIDER/WIDER_train/images/0--Parade/0_Parade_Parade_0_476.jpg',
 '/mnt/Data/Workspace/ml/datasets/WIDER/WIDER_train/images/0--Parade/0_Parade_marchingband_1_12.j

In [ ]:
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

# 3: Set the batch size.

batch_size = 32 # Change the batch size if you like, or if you run into GPU memory issues.

# 4: Set the image transformations for pre-processing and data augmentation options.

# For the training generator:
ssd_data_augmentation = SSDDataAugmentation(img_height=300,
                                            img_width=300,
                                            background=[123, 117, 104])

# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.

# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [model.get_layer('conv4_3_norm_mbox_conf').output_shape[1:3],
                   model.get_layer('fc7_mbox_conf').output_shape[1:3],
                   model.get_layer('conv6_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv7_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv8_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv9_2_mbox_conf').output_shape[1:3]]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.

train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[ssd_data_augmentation],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,
                                                      resize],
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))